<a href="https://colab.research.google.com/github/Weverson3000/Text-Mining-and-Analysis/blob/main/textual_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AS01: Pré-processamento Textual


*  Curso: Engenharia da Computação
*  Materia: Tópicos em Computação III
*  Aluno: Weverson Euzebio Forbes Silva

## Carregamento do arquivo Shakespeare.txt de um link publico




In [ ]:
import gdown

# URL compartilhada do Google Drive
url = "https://drive.google.com/uc?id=12e48l57FEPwaKXpo_aeB2tBy4kan4U8V"

# Caminho para salvar o arquivo localmente
output = "arquivo_baixado.txt"

# Baixando o arquivo
gdown.download(url, output, quiet=False)

# Lendo o conteúdo do arquivo
with open(output, 'r') as file:
    conteudo = file.read()

#print(conteudo)


Downloading...
From: https://drive.google.com/uc?id=12e48l57FEPwaKXpo_aeB2tBy4kan4U8V
To: /content/arquivo_baixado.txt
100%|██████████| 100k/100k [00:00<00:00, 58.5MB/s]


# 1. Normalização

In [ ]:
import re
import unicodedata

# Função que remove acentos e diacríticos
def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

# Função que normalização o texto
def normalizar_texto(texto):
    texto = texto.lower()  # Lower case
    texto = remover_acentos(texto)  # Remover acentos e diacríticos
    texto = re.sub(r'\b[a-z]{1,2}\.\b', '', texto)  # Canonicalizar acrônimos (ex: Ms. -> Ms)
    texto = re.sub(r'\$', 'dollar', texto)  # Canonicalizar moeda
    texto = re.sub(r'\d{1,2}/\d{1,2}/\d{4}', 'data', texto)  # Canonicalizar data
    texto = re.sub(r'-', ' ', texto)  # Canonicalizar palavras com hífen
    texto = re.sub(r'[^\w\s]', '', texto)  # Remover pontuação
    texto = re.sub(r'[^A-Za-z0-9\s]', '', texto)  # Remover caracteres especiais
    return texto

# Texto de entrada para teste
#texto = """It's true, Ms. Martha Töpfer! $3.00 on 3/21/2023 in cash for an ice-cream in the U.S. market? :-( #Truth"""

texto = conteudo

# Aplicando a normalização
texto_normalizado = normalizar_texto(texto)

# Salvar o arquivo normalizado
with open('Shakespeare_Normalized.txt', 'w', encoding='utf-8') as file:
    file.write(texto_normalizado)


## Tokenização

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, TreebankWordTokenizer, WordPunctTokenizer, TweetTokenizer, MWETokenizer
from textblob import TextBlob
import spacy
from gensim.utils import tokenize as gensim_tokenize
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Carregar o texto normalizado
with open('Shakespeare_Normalized.txt', 'r', encoding='utf-8') as file:
    texto_normalizado = file.read()

# Definir algumas expressões pre determinadas (MWE) para o MWETokenizer
mwe_tokenizer = MWETokenizer([('New', 'York'), ('San', 'Francisco'), ('et', 'al')], separator='_')

# Tokenizações
tokenizations = {
    "Whitespace": texto_normalizado.split(),
    "NLTK Word": word_tokenize(texto_normalizado),
    "NLTK Treebank": TreebankWordTokenizer().tokenize(texto_normalizado),
    "NLTK Word Punctuation": WordPunctTokenizer().tokenize(texto_normalizado),
    "NLTK Tweet": TweetTokenizer().tokenize(texto_normalizado),
    "NLTK MWE": mwe_tokenizer.tokenize(word_tokenize(texto_normalizado)),
    "TextBlob": TextBlob(texto_normalizado).words,
    "spaCy": [token.text for token in spacy.load('en_core_web_sm')(texto_normalizado)],
    "Gensim": list(gensim_tokenize(texto_normalizado)),
    "Keras": text_to_word_sequence(texto_normalizado)
}

# Salvar as tokenizações
for i, (name, tokens) in enumerate(tokenizations.items(), 1):
    with open(f'Shakespeare_Normalized_Tokenized{i:02}.txt', 'w', encoding='utf-8') as file:
        file.write(' '.join(tokens))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Stop-Words Removal

In [ ]:
import nltk
from nltk.corpus import stopwords

# Baixar os stop-words
nltk.download('stopwords')

# Carregar os stop-words
stop_words = set(stopwords.words('english'))

# Carregar a tokenização desejada usando so o NLTK Word Tokenizer, indicado no enunciado da atividade
with open('Shakespeare_Normalized_Tokenized02.txt', 'r', encoding='utf-8') as file:
    tokens = file.read().split()

# Remover stop-words
tokens_sem_stopwords = [token for token in tokens if token.lower() not in stop_words]

# Salvar o arquivo sem stop-words
with open('Shakespeare_Normalized_Tokenized_StopWord.txt', 'w', encoding='utf-8') as file:
    file.write(' '.join(tokens_sem_stopwords))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Text Lemmatization

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

# Baixar o recurso wordnet
nltk.download('wordnet')

# Inicializar o lematizador
lemmatizer = WordNetLemmatizer()

# Carregar o texto sem stop-words
with open('Shakespeare_Normalized_Tokenized_StopWord.txt', 'r', encoding='utf-8') as file:
    tokens_sem_stopwords = file.read().split()

# Lematizar o texto
tokens_lematizados = [lemmatizer.lemmatize(token) for token in tokens_sem_stopwords]

# Salvar o arquivo lematizado
with open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized.txt', 'w', encoding='utf-8') as file:
    file.write(' '.join(tokens_lematizados))


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from nltk.stem import PorterStemmer, SnowballStemmer

# Inicializar os stemmers
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer('english')

# Carregar o texto lematizado
with open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized.txt', 'r', encoding='utf-8') as file:
    tokens_lematizados = file.read().split()

# Aplicar Porter Stemmer
tokens_porter = [porter_stemmer.stem(token) for token in tokens_lematizados]
with open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized_Stemming01.txt', 'w', encoding='utf-8') as file:
    file.write(' '.join(tokens_porter))

# Aplicar Snowball Stemmer
tokens_snowball = [snowball_stemmer.stem(token) for token in tokens_lematizados]
with open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized_Stemming02.txt', 'w', encoding='utf-8') as file:
    file.write(' '.join(tokens_snowball))


## Análise do Vocabulário

In [ ]:
import csv
from collections import Counter

# Função que salvar a análise do vocabulário
def salvar_analise_vocabulario(tokens, nome_arquivo):
    vocabulario = Counter(tokens)
    with open(nome_arquivo, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Token", "Ocorrências", "Tamanho"])
        #print(chegou ate 1)
        for token, count in vocabulario.items():
            writer.writerow([token, count, len(token)])
            #print(chegou ate 2)


# Carregar e analisar vocabulários
tokens_lematizados = open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized.txt', 'r', encoding='utf-8').read().split()
tokens_porter = open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized_Stemming01.txt', 'r', encoding='utf-8').read().split()
tokens_snowball = open('Shakespeare_Normalized_Tokenized_StopWord_Lemmatized_Stemming02.txt', 'r', encoding='utf-8').read().split()

# Salvar os CSVs
salvar_analise_vocabulario(tokens_lematizados, 'Shakespeare_Vocabulary_Lemmatized.csv')
salvar_analise_vocabulario(tokens_porter, 'Shakespeare_Vocabulary_Porter.csv')
salvar_analise_vocabulario(tokens_snowball, 'Shakespeare_Vocabulary_Snowball.csv')
#print(chegou ate 2)

# Análise comparativa
def analisar_vocabulario(vocabulario):
    num_tokens = len(vocabulario)
    ocorrencias = sum(vocabulario.values())
    tamanho_medio = sum(len(token) for token in vocabulario) / num_tokens
    #print(chegou ate 3)
    return num_tokens, ocorrencias / num_tokens, tamanho_medio

analise_lematizada = analisar_vocabulario(Counter(tokens_lematizados))
analise_porter = analisar_vocabulario(Counter(tokens_porter))
analise_snowball = analisar_vocabulario(Counter(tokens_snowball))


# Salvar análise comparativa
with open('Shakespeare_Vocabulary_Analysis.txt', 'w', encoding='utf-8') as file:
    file.write("Lemmatizer:\n")
    file.write(f"Vocabulário: {analise_lematizada[0]}\n")
    file.write(f"Média de ocorrências: {analise_lematizada[1]:.2f}\n")
    file.write(f"Tamanho médio dos tokens: {analise_lematizada[2]:.2f}\n\n")
    #print(chegou ate 4)

    file.write("Porter Stemmer:\n")
    file.write(f"Vocabulário: {analise_porter[0]}\n")
    file.write(f"Média de ocorrências: {analise_porter[1]:.2f}\n")
    file.write(f"Tamanho médio dos tokens: {analise_porter[2]:.2f}\n\n")

    file.write("Snowball Stemmer:\n")
    file.write(f"Vocabulário: {analise_snowball[0]}\n")
    file.write(f"Média de ocorrências: {analise_snowball[1]:.2f}\n")
    file.write(f"Tamanho médio dos tokens: {analise_snowball[2]:.2f}\n")
    #print(chegou ate 5)



## "Os arquivos gerados estaram localizados no seguinte diretório: /content/sample_data."

# Referências bibliográficas:

Manning, Raghavan & Schütze, 2008

Slides e Videos:
LC02: Google Colaboratory,
LC04: Normalização de Texto,
LC05: Tokenização,
LC06: Stop-Words,
LC07: Lematização,
LC08: Stemming

**Biblioteca `gdown`:** A biblioteca `gdown` para download de arquivos do Google Drive. **Referência**: https://pypi.org/project/gdown/

**Documentação Python (para funçao `open()`):** A função `open()` Permite abrir e ler arquivos de texto em python. **Referência**: https://docs.python.org/3/library/functions.html#open

Documentação biblioteca nltk para tokenização, remoção de stopwords, lematização e stemming:

Documentação Bird, S., Klein, E., & Loper, E. (2009). Natural Language Processing with Python. O'Reilly Media, Inc.
Biblioteca spacy para tokenização:


Documentação. https://keras.io.
Uso do TextBlob para tokenização e processamento de texto:

Documentação https://textblob.readthedocs.io. Biblioteca unicodedata para remoção de acentos e diacríticos:

Tokenização usando nltk:

Como tokenizar uma frase em NLTK?
https://stackoverflow.com/questions/15547409/
Remoção de stopwords com nltk:

Removing stop words from a string in Python
https://stackoverflow.com/questions/5486337/
Lematização usando WordNetLemmatizer:

Qual é a diferença entre stemming e lemmatization?
https://stackoverflow.com/questions/437489
Uso do PorterStemmer para stemming:

Porter stemmer in NLTK
https://stackoverflow.com/questions/36182502/



